# Finnt- Feature Engineering & Data Preprocessing

**Advanced Feature Engineering for Personalized Loan Offer Engine**

This notebook transforms the raw Lending Club dataset into a clean, feature-rich dataset ready for machine learning. We'll engineer our dual target variables and create robust features for our optimization models.

## Objectives:
1. **Target Engineering**: Create `is_default` for P(Default) model
2. **Data Cleaning**: Handle missing values and data quality issues
3. **Feature Transformation**: Convert categorical and date features
4. **Data Type Optimization**: Ensure proper data types for modeling
5. **MLOps Integration**: Version the processed dataset with DVC

**Date**: July 8, 2025  
**Phase**: 2 - Feature Engineering

## 1. Environment Setup

Import essential libraries for data manipulation, feature engineering, and date processing.

In [1]:
# Core data manipulation
import pandas as pd
import numpy as np
from datetime import datetime
import re

# Visualization for data exploration
import matplotlib.pyplot as plt
import seaborn as sns

# File handling
import os
from pathlib import Path

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

print("Libraries loaded successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

# Set random seed for reproducibility
np.random.seed(42)

Libraries loaded successfully!
Pandas version: 2.3.1
NumPy version: 2.0.2


## 2. Load Raw Data

Load the original Lending Club dataset from our DVC-tracked data directory for comprehensive feature engineering.

In [2]:
# Load the raw dataset
data_path = '../data/raw/loan.csv'
print(f"Loading raw dataset from: {data_path}")

# Read CSV with optimized settings
df_raw = pd.read_csv(data_path, low_memory=False)

print(f"Raw dataset loaded successfully!")
print(f"Original dataset shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")
print(f"Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Create a working copy for feature engineering
df = df_raw.copy()
print(f"Working copy created for feature engineering")

Loading raw dataset from: ../data/raw/loan.csv
Raw dataset loaded successfully!
Original dataset shape: 2,260,668 rows × 145 columns
Raw dataset loaded successfully!
Original dataset shape: 2,260,668 rows × 145 columns
Memory usage: 5942.3 MB
Memory usage: 5942.3 MB
Working copy created for feature engineering
Working copy created for feature engineering


## 3. Target Variable Engineering

**Critical Step**: Create our target variables for the dual-model approach. 

In [3]:
# First, examine the unique loan statuses in our dataset
print("LOAN STATUS ANALYSIS")
print("=" * 50)
loan_status_counts = df['loan_status'].value_counts()
print("Current loan statuses in dataset:")
for status, count in loan_status_counts.items():
    pct = (count / len(df)) * 100
    print(f"  {status}: {count:,} ({pct:.2f}%)")

print(f"\nTotal unique statuses: {df['loan_status'].nunique()}")

# Define default events for our risk model
default_statuses = [
    'Charged Off',
    'Default', 
    'Does not meet the credit policy. Status:Charged Off',
    'Late (31-120 days)'
]

print(f"\nDEFINING DEFAULT EVENTS:")
print("The following loan statuses will be classified as defaults:")
for status in default_statuses:
    if status in loan_status_counts:
        count = loan_status_counts[status]
        pct = (count / len(df)) * 100
        print(f"  {status}: {count:,} ({pct:.2f}%)")
    else:
        print(f"  {status}: Not found in dataset")

LOAN STATUS ANALYSIS
Current loan statuses in dataset:
  Fully Paid: 1,041,952 (46.09%)
  Current: 919,695 (40.68%)
  Charged Off: 261,655 (11.57%)
  Late (31-120 days): 21,897 (0.97%)
  In Grace Period: 8,952 (0.40%)
  Late (16-30 days): 3,737 (0.17%)
  Does not meet the credit policy. Status:Fully Paid: 1,988 (0.09%)
  Does not meet the credit policy. Status:Charged Off: 761 (0.03%)
  Default: 31 (0.00%)

Total unique statuses: 9

DEFINING DEFAULT EVENTS:
The following loan statuses will be classified as defaults:
  Charged Off: 261,655 (11.57%)
  Default: 31 (0.00%)
  Does not meet the credit policy. Status:Charged Off: 761 (0.03%)
  Late (31-120 days): 21,897 (0.97%)


In [4]:
# Create the binary default target variable
df['is_default'] = df['loan_status'].isin(default_statuses).astype(int)

# Analyze the target variable distribution
default_rate = df['is_default'].mean() * 100
print(f"\nTARGET VARIABLE: is_default")
print("=" * 50)
print(f"Overall Default Rate: {default_rate:.2f}%")
print(f"Non-Default Rate: {100-default_rate:.2f}%")

print(f"\nClass Distribution:")
target_distribution = df['is_default'].value_counts().sort_index()
for value, count in target_distribution.items():
    label = "Default" if value == 1 else "Non-Default"
    pct = (count / len(df)) * 100
    print(f"  {value} ({label}): {count:,} ({pct:.2f}%)")

# Check for class imbalance
class_ratio = target_distribution[0] / target_distribution[1]
print(f"\nClass Balance Analysis:")
print(f"Non-Default to Default Ratio: {class_ratio:.1f}:1")
if class_ratio > 10:
    print("Significant class imbalance detected - will need balancing techniques")
elif class_ratio > 5:
    print("Moderate class imbalance - consider balancing techniques")
else:
    print("Reasonable class balance for modeling")


TARGET VARIABLE: is_default
Overall Default Rate: 12.58%
Non-Default Rate: 87.42%

Class Distribution:
  0 (Non-Default): 1,976,324 (87.42%)
  1 (Default): 284,344 (12.58%)

Class Balance Analysis:
Non-Default to Default Ratio: 7.0:1
Moderate class imbalance - consider balancing techniques


### P(Acceptance) Target Strategy

**Important Note**: All loans in this historical dataset were **implicitly accepted** by customers (since they exist in the dataset). Therefore, creating a simple binary acceptance target would result in all `1`s, which isn't useful for modeling.

**Our Strategy for P(Acceptance) Modeling**:
1. **Phase 3**: We'll use interest rate sensitivity analysis to create synthetic acceptance scenarios
2. **Feature Engineering**: Derive proxy variables like `requested_vs_funded_ratio` to understand acceptance patterns
3. **External Data**: In production, we'd collect customer response data to loan offers at different rates
4. **Simulation**: Create acceptance probability curves based on rate spreads and customer segments

For now, we focus on the **P(Default) model** which has a clear, well-defined target variable (`is_default`).

## 4. Initial Feature Selection & Data Leakage Prevention

**Critical MLOps Step**: Remove features that would not be available at loan application time or have excessive missing values. This prevents data leakage and ensures our model is realistic.

In [5]:
# First, identify columns with high missing values (>40%)
print("MISSING VALUE ANALYSIS")
print("=" * 50)

missing_pct = (df.isnull().sum() / len(df)) * 100
high_missing_cols = missing_pct[missing_pct > 40].sort_values(ascending=False)

print(f"Columns with >40% missing data ({len(high_missing_cols)} total):")
for col, pct in high_missing_cols.items():
    print(f"  {col}: {pct:.1f}% missing")

# Define columns that cause data leakage (not available at application time)
leakage_columns = [
    # Loan identifiers (not predictive)
    'id', 'member_id', 'url',
    
    # Geographic data (too granular, privacy concerns)
    'zip_code',
    
    # Target variable (we have our engineered version)
    'loan_status',
    
    # Payment history (not available at application time)
    'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
    'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
    'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
    
    # Policy codes and internal flags
    'policy_code', 'application_type',
    
    # Post-loan information
    'hardship_flag', 'disbursement_method', 'debt_settlement_flag'
]

print(f"\nDATA LEAKAGE PREVENTION")
print("=" * 50)
print("Columns to drop (would not be available at loan application time):")

available_leakage_cols = [col for col in leakage_columns if col in df.columns]
for col in available_leakage_cols:
    print(f"  {col}")

missing_leakage_cols = [col for col in leakage_columns if col not in df.columns]
if missing_leakage_cols:
    print(f"\nLeakage columns not found in dataset:")
    for col in missing_leakage_cols:
        print(f"  {col}")

# Combine all columns to drop
high_missing_col_names = high_missing_cols.index.tolist()
columns_to_drop = list(set(available_leakage_cols + high_missing_col_names))

# CRITICAL: Ensure is_default is NEVER dropped (our target variable)
if 'is_default' in columns_to_drop:
    columns_to_drop.remove('is_default')
    print(f"\nis_default protected from being dropped (our target variable)")

print(f"\nSUMMARY")
print("=" * 30)
print(f"Original columns: {len(df.columns)}")
print(f"Columns to drop: {len(columns_to_drop)}")
print(f"Remaining columns: {len(df.columns) - len(columns_to_drop)}")
print(f"is_default protected: {'is_default' in df.columns and 'is_default' not in columns_to_drop}")

MISSING VALUE ANALYSIS
Columns with >40% missing data (46 total):
  id: 100.0% missing
  url: 100.0% missing
  member_id: 100.0% missing
  orig_projected_additional_accrued_interest: 99.6% missing
  hardship_length: 99.5% missing
  hardship_reason: 99.5% missing
  hardship_status: 99.5% missing
  deferral_term: 99.5% missing
  hardship_amount: 99.5% missing
  hardship_start_date: 99.5% missing
  hardship_end_date: 99.5% missing
  payment_plan_start_date: 99.5% missing
  hardship_loan_status: 99.5% missing
  hardship_dpd: 99.5% missing
  hardship_payoff_balance_amount: 99.5% missing
  hardship_last_payment_amount: 99.5% missing
  hardship_type: 99.5% missing
  debt_settlement_flag_date: 98.5% missing
  settlement_status: 98.5% missing
  settlement_date: 98.5% missing
  settlement_amount: 98.5% missing
  settlement_percentage: 98.5% missing
  settlement_term: 98.5% missing
  sec_app_mths_since_last_major_derog: 98.4% missing
  sec_app_revol_util: 95.3% missing
  revol_bal_joint: 95.2% mi

In [6]:
# Drop the identified columns
print("DROPPING COLUMNS")
print("=" * 50)

initial_shape = df.shape
df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')
final_shape = df_cleaned.shape

print(f"Dropped {len(columns_to_drop)} columns:")
for col in sorted(columns_to_drop):
    if col in df.columns:
        print(f"  ✓ {col}")

print(f"\nDataset shape change:")
print(f"  Before: {initial_shape[0]:,} rows × {initial_shape[1]} columns")
print(f"  After:  {final_shape[0]:,} rows × {final_shape[1]} columns")
print(f"  Columns reduced by: {initial_shape[1] - final_shape[1]}")

# Update our working dataframe
df = df_cleaned.copy()
print(f"\n✅ Feature selection complete. Working with {len(df.columns)} columns.")

DROPPING COLUMNS
Dropped 63 columns:
  ✓ annual_inc_joint
  ✓ application_type
  ✓ collection_recovery_fee
  ✓ debt_settlement_flag
  ✓ debt_settlement_flag_date
  ✓ deferral_term
  ✓ desc
  ✓ disbursement_method
  ✓ dti_joint
  ✓ hardship_amount
  ✓ hardship_dpd
  ✓ hardship_end_date
  ✓ hardship_flag
  ✓ hardship_last_payment_amount
  ✓ hardship_length
  ✓ hardship_loan_status
  ✓ hardship_payoff_balance_amount
  ✓ hardship_reason
  ✓ hardship_start_date
  ✓ hardship_status
  ✓ hardship_type
  ✓ id
  ✓ il_util
  ✓ last_credit_pull_d
  ✓ last_pymnt_amnt
  ✓ last_pymnt_d
  ✓ loan_status
  ✓ member_id
  ✓ mths_since_last_delinq
  ✓ mths_since_last_major_derog
  ✓ mths_since_last_record
  ✓ mths_since_rcnt_il
  ✓ mths_since_recent_bc_dlq
  ✓ mths_since_recent_revol_delinq
  ✓ next_pymnt_d
  ✓ orig_projected_additional_accrued_interest
  ✓ payment_plan_start_date
  ✓ policy_code
  ✓ recoveries
  ✓ revol_bal_joint
  ✓ sec_app_chargeoff_within_12_mths
  ✓ sec_app_collections_12_mths_ex_med


### Why Data Leakage Prevention is Critical

**Data Leakage** occurs when information that would not be available at prediction time is used during model training. This creates artificially high model performance that doesn't translate to real-world scenarios.

**Examples of Leakage in Loan Default Prediction**:
- **`total_pymnt`**: Total payments made - only known after loan completion
- **`last_pymnt_d`**: Last payment date - reveals loan performance over time  
- **`recoveries`**: Amount recovered post-default - only known after default occurs
- **`collection_recovery_fee`**: Fees from collections - indicates default already happened

**Impact**: Models trained with leakage can achieve 99%+ accuracy in testing but fail completely in production because the "future" information isn't available when making real predictions.

**Our Approach**: Only use information available at loan origination time, ensuring our model is realistic.

## 5. Data Type Transformations

Convert string-based features to appropriate numeric formats for machine learning models.

In [7]:
# 5.1 Convert Interest Rate from percentage string to float
print("INTEREST RATE TRANSFORMATION")
print("=" * 50)

if 'int_rate' in df.columns:
    # Check current format
    print("Current int_rate format (first 10 values):")
    print(df['int_rate'].head(10).tolist())
    
    # Convert percentage string to float
    # Remove '%' and convert to float
    df['int_rate'] = df['int_rate'].astype(str).str.replace('%', '').astype(float)
    
    print(f"\n✅ Converted int_rate from percentage string to float")
    print("New int_rate format (first 10 values):")
    print(df['int_rate'].head(10).tolist())
    
    print(f"\nInterest Rate Statistics:")
    print(f"  Mean: {df['int_rate'].mean():.2f}%")
    print(f"  Median: {df['int_rate'].median():.2f}%")
    print(f"  Range: {df['int_rate'].min():.2f}% - {df['int_rate'].max():.2f}%")
else:
    print("⚠️ int_rate column not found in dataset")

INTEREST RATE TRANSFORMATION
Current int_rate format (first 10 values):
[13.56, 18.94, 17.97, 18.94, 16.14, 15.02, 17.97, 13.56, 17.97, 14.47]

✅ Converted int_rate from percentage string to float
New int_rate format (first 10 values):
[13.56, 18.94, 17.97, 18.94, 16.14, 15.02, 17.97, 13.56, 17.97, 14.47]

Interest Rate Statistics:
  Mean: 13.09%
  Median: 12.62%
  Range: 5.31% - 30.99%

✅ Converted int_rate from percentage string to float
New int_rate format (first 10 values):
[13.56, 18.94, 17.97, 18.94, 16.14, 15.02, 17.97, 13.56, 17.97, 14.47]

Interest Rate Statistics:
  Mean: 13.09%
  Median: 12.62%
  Range: 5.31% - 30.99%


In [8]:
# 5.2 Convert Term from string to integer
print("\nTERM TRANSFORMATION")
print("=" * 50)

if 'term' in df.columns:
    # Check current format
    print("Current term format (unique values):")
    print(df['term'].value_counts())
    
    # Extract numeric value from term string (e.g., " 36 months" -> 36)
    df['term'] = df['term'].str.extract('(\d+)').astype(int)
    
    print(f"\n✅ Converted term from string to integer")
    print("New term format (unique values):")
    print(df['term'].value_counts())
    
    print(f"\nTerm Statistics:")
    print(f"  Mean: {df['term'].mean():.1f} months")
    print(f"  Most common: {df['term'].mode()[0]} months")
else:
    print("⚠️ term column not found in dataset")


TERM TRANSFORMATION
Current term format (unique values):
term
36 months    1609754
60 months     650914
Name: count, dtype: int64

✅ Converted term from string to integer
New term format (unique values):
term
36    1609754
60     650914
Name: count, dtype: int64

Term Statistics:
  Mean: 42.9 months
  Most common: 36 months

✅ Converted term from string to integer
New term format (unique values):
term
36    1609754
60     650914
Name: count, dtype: int64

Term Statistics:
  Mean: 42.9 months
  Most common: 36 months


In [9]:
# 5.3 Convert Employment Length to ordinal numeric feature
print("\nEMPLOYMENT LENGTH TRANSFORMATION")
print("=" * 50)

if 'emp_length' in df.columns:
    # Check current format
    print("Current emp_length format (value counts):")
    emp_length_counts = df['emp_length'].value_counts(dropna=False)
    print(emp_length_counts)
    
    def convert_emp_length(emp_str):
        """Convert employment length string to numeric years"""
        if pd.isna(emp_str):
            return np.nan
        
        emp_str = str(emp_str).strip().lower()
        
        if '< 1 year' in emp_str or 'less than 1' in emp_str:
            return 0
        elif '10+ years' in emp_str:
            return 10
        else:
            # Extract number from strings like "2 years", "5 years", etc.
            match = re.search(r'(\d+)', emp_str)
            if match:
                return int(match.group(1))
            else:
                return np.nan
    
    # Apply the conversion
    df['emp_length_years'] = df['emp_length'].apply(convert_emp_length)
    
    print(f"\n✅ Converted emp_length to numeric emp_length_years")
    print("New emp_length_years distribution:")
    print(df['emp_length_years'].value_counts(dropna=False).sort_index())
    
    print(f"\nEmployment Length Statistics:")
    print(f"  Mean: {df['emp_length_years'].mean():.1f} years")
    print(f"  Median: {df['emp_length_years'].median():.1f} years")
    print(f"  Missing values: {df['emp_length_years'].isnull().sum():,}")
    
    # Drop the original emp_length column
    df = df.drop('emp_length', axis=1)
    print(f"  Dropped original emp_length column")
    
else:
    print("⚠️ emp_length column not found in dataset")


EMPLOYMENT LENGTH TRANSFORMATION
Current emp_length format (value counts):
emp_length
10+ years    748005
2 years      203677
< 1 year     189988
3 years      180753
1 year       148403
NaN          146907
5 years      139698
4 years      136605
6 years      102628
7 years       92695
8 years       91914
9 years       79395
Name: count, dtype: int64

✅ Converted emp_length to numeric emp_length_years
New emp_length_years distribution:
emp_length_years
0.0     189988
1.0     148403
2.0     203677
3.0     180753
4.0     136605
5.0     139698
6.0     102628
7.0      92695
8.0      91914
9.0      79395
10.0    748005
NaN     146907
Name: count, dtype: int64

Employment Length Statistics:
  Mean: 5.9 years
  Median: 6.0 years
  Missing values: 146,907

✅ Converted emp_length to numeric emp_length_years
New emp_length_years distribution:
emp_length_years
0.0     189988
1.0     148403
2.0     203677
3.0     180753
4.0     136605
5.0     139698
6.0     102628
7.0      92695
8.0      91914
9.0

## 6. Date Feature Engineering

Create meaningful features from date columns to capture temporal patterns and credit history length.

In [10]:
# 6.1 Convert date columns and create credit history length
print("DATE FEATURE ENGINEERING")
print("=" * 50)

# Check which date columns we have
date_columns = ['earliest_cr_line', 'issue_d']
available_date_cols = [col for col in date_columns if col in df.columns]
print(f"Available date columns: {available_date_cols}")

if 'earliest_cr_line' in df.columns and 'issue_d' in df.columns:
    print(f"\nSample date formats:")
    print(f"  earliest_cr_line: {df['earliest_cr_line'].dropna().iloc[0:3].tolist()}")
    print(f"  issue_d: {df['issue_d'].dropna().iloc[0:3].tolist()}")
    
    # Convert to datetime
    try:
        df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y', errors='coerce')
        df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y', errors='coerce')
        
        print(f"\n✅ Successfully converted date columns to datetime")
        
        # Create credit history length in years
        df['credit_history_years'] = (df['issue_d'] - df['earliest_cr_line']).dt.days / 365.25
        
        print(f"\nCredit History Statistics:")
        print(f"  Mean: {df['credit_history_years'].mean():.1f} years")
        print(f"  Median: {df['credit_history_years'].median():.1f} years")
        print(f"  Range: {df['credit_history_years'].min():.1f} - {df['credit_history_years'].max():.1f} years")
        print(f"  Missing values: {df['credit_history_years'].isnull().sum():,}")
        
        # Clean up - drop original date columns after feature extraction
        df = df.drop(['earliest_cr_line', 'issue_d'], axis=1)
        print(f"\nDropped original date columns (retained engineered features)")
        
    except Exception as e:
        print(f"⚠️ Error converting dates: {e}")
        print("Skipping date feature engineering")
        
elif 'earliest_cr_line' in df.columns:
    print("\n⚠️ Only earliest_cr_line available - cannot create credit history length")
else:
    print("\n⚠️ Required date columns not found in dataset")

DATE FEATURE ENGINEERING
Available date columns: ['earliest_cr_line', 'issue_d']

Sample date formats:
  earliest_cr_line: ['Apr-2001', 'Jun-1987', 'Apr-2011']
  issue_d: ['Dec-2018', 'Dec-2018', 'Dec-2018']

✅ Successfully converted date columns to datetime

Credit History Statistics:
  Mean: 16.4 years
  Median: 14.8 years
  Range: 0.5 - 83.3 years
  Missing values: 29

✅ Successfully converted date columns to datetime

Credit History Statistics:
  Mean: 16.4 years
  Median: 14.8 years
  Range: 0.5 - 83.3 years
  Missing values: 29

Dropped original date columns (retained engineered features)

Dropped original date columns (retained engineered features)


## 7. Handling Remaining Missing Values

Apply robust imputation strategies for the remaining missing values in our cleaned dataset.

In [11]:
# Analyze remaining missing values after feature engineering
print("REMAINING MISSING VALUES ANALYSIS")
print("=" * 50)

missing_after_engineering = df.isnull().sum()
missing_cols = missing_after_engineering[missing_after_engineering > 0].sort_values(ascending=False)

if len(missing_cols) > 0:
    print(f"Columns with missing values ({len(missing_cols)} total):")
    for col, count in missing_cols.items():
        pct = (count / len(df)) * 100
        print(f"  {col}: {count:,} ({pct:.1f}%)")
    
    # Separate numeric and categorical columns for different imputation strategies
    numeric_cols_with_missing = []
    categorical_cols_with_missing = []
    
    for col in missing_cols.index:
        if df[col].dtype in ['int64', 'float64']:
            numeric_cols_with_missing.append(col)
        else:
            categorical_cols_with_missing.append(col)
    
    print(f"\nMissing Value Categories:")
    print(f"  Numeric columns: {numeric_cols_with_missing}")
    print(f"  Categorical columns: {categorical_cols_with_missing}")
    
else:
    print("✅ No missing values remaining in the dataset!")

REMAINING MISSING VALUES ANALYSIS
Columns with missing values (64 total):
  all_util: 866,348 (38.3%)
  open_acc_6m: 866,130 (38.3%)
  inq_last_12m: 866,130 (38.3%)
  total_cu_tl: 866,130 (38.3%)
  inq_fi: 866,129 (38.3%)
  open_il_12m: 866,129 (38.3%)
  open_act_il: 866,129 (38.3%)
  open_rv_12m: 866,129 (38.3%)
  open_rv_24m: 866,129 (38.3%)
  max_bal_bc: 866,129 (38.3%)
  open_il_24m: 866,129 (38.3%)
  total_bal_il: 866,129 (38.3%)
  mths_since_recent_inq: 295,435 (13.1%)
  emp_title: 166,969 (7.4%)
  num_tl_120dpd_2m: 153,657 (6.8%)
  emp_length_years: 146,907 (6.5%)
  mo_sin_old_il_acct: 139,071 (6.2%)
  bc_util: 76,071 (3.4%)
  percent_bc_gt_75: 75,379 (3.3%)
  bc_open_to_buy: 74,935 (3.3%)
  mths_since_recent_bc: 73,412 (3.2%)
  pct_tl_nvr_dlq: 70,431 (3.1%)
  avg_cur_bal: 70,346 (3.1%)
  num_rev_accts: 70,277 (3.1%)
  mo_sin_old_rev_tl_op: 70,277 (3.1%)
  mo_sin_rcnt_rev_tl_op: 70,277 (3.1%)
  num_tl_90g_dpd_24m: 70,276 (3.1%)
  num_tl_30dpd: 70,276 (3.1%)
  num_rev_tl_bal_gt_0

In [12]:
# Apply imputation strategies
print("\nAPPLYING IMPUTATION STRATEGIES")
print("=" * 50)

if len(missing_cols) > 0:
    # For numeric columns: use median (robust to outliers)
    if numeric_cols_with_missing:
        print("Numeric columns - using MEDIAN imputation:")
        for col in numeric_cols_with_missing:
            original_missing = df[col].isnull().sum()
            median_value = df[col].median()
            df[col] = df[col].fillna(median_value)
            print(f"  ✓ {col}: filled {original_missing:,} missing values with {median_value:.2f}")
    
    # For categorical columns: use mode (most frequent value)
    if categorical_cols_with_missing:
        print(f"\nCategorical columns - using MODE imputation:")
        for col in categorical_cols_with_missing:
            original_missing = df[col].isnull().sum()
            mode_value = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
            df[col] = df[col].fillna(mode_value)
            print(f"  ✓ {col}: filled {original_missing:,} missing values with '{mode_value}'")
    
    # Verify no missing values remain
    final_missing = df.isnull().sum().sum()
    print(f"\n✅ Imputation complete. Total missing values: {final_missing}")
    
else:
    print("No imputation needed - dataset already complete!")

print(f"\nWhy MEDIAN over MEAN for numeric features?")
print(f"  • Median is robust to outliers and skewed distributions")
print(f"  • Mean can be heavily influenced by extreme values")
print(f"  • Financial data often has outliers (high earners, large loans)")
print(f"  • Median preserves the central tendency better")


APPLYING IMPUTATION STRATEGIES
Numeric columns - using MEDIAN imputation:
  ✓ all_util: filled 866,348 missing values with 58.00
  ✓ open_acc_6m: filled 866,130 missing values with 1.00
  ✓ inq_last_12m: filled 866,130 missing values with 1.00
  ✓ total_cu_tl: filled 866,130 missing values with 0.00
  ✓ inq_fi: filled 866,129 missing values with 1.00
  ✓ open_il_12m: filled 866,129 missing values with 0.00
  ✓ open_act_il: filled 866,129 missing values with 2.00
  ✓ open_rv_12m: filled 866,129 missing values with 1.00  ✓ open_rv_12m: filled 866,129 missing values with 1.00
  ✓ open_rv_24m: filled 866,129 missing values with 2.00
  ✓ max_bal_bc: filled 866,129 missing values with 4413.00
  ✓ open_il_24m: filled 866,129 missing values with 1.00
  ✓ total_bal_il: filled 866,129 missing values with 23127.00
  ✓ mths_since_recent_inq: filled 295,435 missing values with 5.00
  ✓ num_tl_120dpd_2m: filled 153,657 missing values with 0.00
  ✓ emp_length_years: filled 146,907 missing values wit

## 8. Save Processed Data

Save our cleaned and engineered dataset in an optimized format for downstream modeling.

In [13]:
# Final data summary before saving
print("FINAL DATASET SUMMARY")
print("=" * 50)
print(f"Final dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"Missing values: {df.isnull().sum().sum()}")

# CRITICAL VERIFICATION: Ensure is_default exists in final dataset
if 'is_default' in df.columns:
    print(f"✅ is_default column confirmed in final dataset")
    print(f"   Default rate: {df['is_default'].mean():.3f}")
    print(f"   Class distribution: {df['is_default'].value_counts().to_dict()}")
else:
    print(f"❌ ERROR: is_default column missing from final dataset!")
    print(f"Available columns: {list(df.columns)}")
    raise ValueError("is_default target variable is missing from final dataset")

# Display final feature list
print(f"\nFinal features ({len(df.columns)} total):")
print(f"Columns in final dataset: {list(df.columns)}")
feature_types = df.dtypes.value_counts()
print(f"Data type distribution:")
for dtype, count in feature_types.items():
    print(f"  {dtype}: {count} columns")

# Ensure processed directory exists
processed_dir = Path('../data/processed')
processed_dir.mkdir(exist_ok=True)

# Save to parquet format
output_path = processed_dir / 'processed_loan_data.parquet'
print(f"\nSaving processed dataset to: {output_path}")

df.to_parquet(output_path, index=False, compression='snappy')

# Verify the saved file
if output_path.exists():
    file_size_mb = output_path.stat().st_size / 1024**2
    print(f"✅ Dataset saved successfully!")
    print(f"   File size: {file_size_mb:.1f} MB")
    print(f"   Compression ratio: {df.memory_usage(deep=True).sum() / 1024**2 / file_size_mb:.1f}x")
    
    # Double-check by loading the saved file
    df_verify = pd.read_parquet(output_path)
    if 'is_default' in df_verify.columns:
        print(f"✅ Verified: is_default exists in saved file")
    else:
        print(f"❌ ERROR: is_default missing from saved file!")
else:
    print("❌ Error: File not saved properly")

FINAL DATASET SUMMARY
Final dataset shape: 2,260,668 rows × 82 columns
Memory usage: 2626.7 MB
Missing values: 0
✅ is_default column confirmed in final dataset
   Default rate: 0.126
   Class distribution: {0: 1976324, 1: 284344}

Final features (82 total):
Columns in final dataset: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'home_ownership', 'annual_inc', 'verification_status', 'pymnt_plan', 'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'total_bal_il', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',

Parquet format is used because it is fast, efficient, and preserves data integrity for machine learning workflows.

## 9. Version Processed Data with DVC

**Critical MLOps Step**: Add our processed dataset to DVC tracking for reproducible ML pipelines.

### DVC Versioning Commands

Execute these commands in your terminal to version the processed dataset:

```bash
# 1. Add processed data to DVC tracking
dvc add data/processed/processed_loan_data.parquet

# 2. Add DVC pointer files to Git
git add data/processed/processed_loan_data.parquet.dvc data/processed/.gitignore

# 3. Commit the feature engineering work
git add notebooks/02-Feature-Engineering.ipynb
git commit -m "feat: Phase 2 complete - Feature engineering and data preprocessing

- Created comprehensive feature engineering pipeline
- Engineered binary default target (is_default) 
- Removed data leakage by dropping post-application features
- Converted categorical features: int_rate, term, emp_length
- Created credit_history_years from date features
- Applied robust median imputation for missing values
- Saved optimized dataset in Parquet format (50%+ compression)
- Versioned processed data with DVC for reproducible ML pipeline

Dataset ready for model training: {df.shape[0]:,} rows × {df.shape[1]} features"
```

### What This Achieves:
- **Reproducible Pipeline**: Anyone can recreate the exact processed dataset
- **Data Lineage**: Clear track from raw data to processed features  
- **Version Control**: Changes to preprocessing are tracked and reversible
- **Team Collaboration**: Shared processed datasets across team members
- **Production Ready**: Processed data ready for model training and validation

## Feature Engineering Summary

**Completed Steps:**
1. Created the `is_default` target variable for default prediction.
2. Removed columns with high missing values and those causing data leakage.
3. Transformed features: converted interest rate and term to numeric, employment length to ordinal, and engineered credit history years.
4. Imputed missing values using median for numeric and mode for categorical features.
5. Saved the processed data in Parquet format for efficient storage and modeling.